# 📘 Wczytywanie danych w TensorFlow i Keras

## 1. Wczytywanie CSV wprost do `tf.data.Dataset`

TensorFlow ma wbudowaną funkcję `tf.data.experimental.make_csv_dataset`, która zamienia plik CSV w strumień danych (dataset).

Przykład dla pliku `Data/iris.csv`:

In [1]:
import tensorflow as tf

# Ścieżka do danych
file_path = "../Data/iris.csv"

# Wczytanie CSV jako dataset
dataset = tf.data.experimental.make_csv_dataset(
    file_path,
    batch_size=16,       # rozmiar batcha
    label_name="target", # kolumna etykiety
    num_epochs=1,        # ile razy przejść przez dane
    shuffle=True
)

# Podgląd batcha
for features, label in dataset.take(1):
    print("Cechy:", features)
    print("Etykiety:", label)

2025-11-22 14:29:02.101851: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Cechy: OrderedDict([('sepal length (cm)', <tf.Tensor: shape=(16,), dtype=float32, numpy=
array([6.4, 6.1, 4.7, 6.1, 6.3, 5.6, 5.7, 6.9, 6.2, 7.7, 7.6, 6.8, 5.8,
       7.9, 5.2, 5.4], dtype=float32)>), ('sepal width (cm)', <tf.Tensor: shape=(16,), dtype=float32, numpy=
array([3.2, 2.9, 3.2, 2.8, 3.3, 2.5, 2.8, 3.2, 2.9, 3.8, 3. , 2.8, 2.8,
       3.8, 4.1, 3.4], dtype=float32)>), ('petal length (cm)', <tf.Tensor: shape=(16,), dtype=float32, numpy=
array([5.3, 4.7, 1.6, 4.7, 4.7, 3.9, 4.5, 5.7, 4.3, 6.7, 6.6, 4.8, 5.1,
       6.4, 1.5, 1.7], dtype=float32)>), ('petal width (cm)', <tf.Tensor: shape=(16,), dtype=float32, numpy=
array([2.3, 1.4, 0.2, 1.2, 1.6, 1.1, 1.3, 2.3, 1.3, 2.2, 2.1, 1.4, 2.4,
       2. , 0.1, 0.2], dtype=float32)>)])
Etykiety: tf.Tensor([2. 1. 0. 1. 1. 1. 1. 2. 1. 2. 2. 1. 2. 2. 0. 0.], shape=(16,), dtype=float32)


2025-11-22 14:29:06.039639: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


👉 Zalety: działa dobrze z dużymi plikami, dane wczytywane są leniwie (strumieniowo).
👉 Uwaga: `species` musi być kolumną w CSV.

---

## 2. Wczytanie CSV do Pandas + konwersja do TensorFlow

Często prościej jest najpierw wczytać dane Pandasem:

In [3]:
import pandas as pd
import tensorflow as tf

# Wczytanie Pandasem

df = pd.read_csv("../Data/iris.csv")
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [4]:
df.dtypes

sepal length (cm)    float64
sepal width (cm)     float64
petal length (cm)    float64
petal width (cm)     float64
target               float64
dtype: object

In [3]:
# Oddzielenie X i y
X = df.drop("target", axis=1).values
y = df["target"].astype("category").cat.codes.values  # zamiana na liczby

# Konwersja do tensora
X_tensor = tf.convert_to_tensor(X, dtype=tf.float32)
y_tensor = tf.convert_to_tensor(y, dtype=tf.int32)

print(X_tensor.shape, y_tensor.shape)

(150, 4) (150,)


👉 Ten sposób daje większą kontrolę nad preprocessingiem (np. normalizacja, one-hot encoding).

---

## 3. Użycie Keras `keras.utils` – np. `image_dataset_from_directory`, `text_dataset_from_directory`, `timeseries_dataset_from_array`

Dla CSV nie ma gotowej funkcji, ale można łatwo zamienić Pandas → NumPy → Keras `Dataset`:

In [7]:
from tensorflow import keras
import pandas as pd

# Wczytaj CSV
df = pd.read_csv("../Data/iris.csv")

X = df.drop("target", axis=1).values
y = df["target"].astype("category").cat.codes.values

# Konwersja do datasetu keras
dataset = keras.utils.timeseries_dataset_from_array(
    data=X,
    targets=y,
    sequence_length=1,   # tu akurat nie czasowe, więc 1
    batch_size=16
)

for batch_x, batch_y in dataset.take(1):
    print("X batch:", batch_x.numpy())
    print("y batch:", batch_y.numpy())

X batch: [[[5.1 3.5 1.4 0.2]]

 [[4.9 3.  1.4 0.2]]

 [[4.7 3.2 1.3 0.2]]

 [[4.6 3.1 1.5 0.2]]

 [[5.  3.6 1.4 0.2]]

 [[5.4 3.9 1.7 0.4]]

 [[4.6 3.4 1.4 0.3]]

 [[5.  3.4 1.5 0.2]]

 [[4.4 2.9 1.4 0.2]]

 [[4.9 3.1 1.5 0.1]]

 [[5.4 3.7 1.5 0.2]]

 [[4.8 3.4 1.6 0.2]]

 [[4.8 3.  1.4 0.1]]

 [[4.3 3.  1.1 0.1]]

 [[5.8 4.  1.2 0.2]]

 [[5.7 4.4 1.5 0.4]]]
y batch: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


2025-11-22 14:31:41.405699: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


## 📚 Zbiory danych w `keras.datasets`

W pakiecie znajdziesz m.in.:

* **MNIST** – ręcznie pisane cyfry (0–9, grayscale, 28×28)
* **Fashion-MNIST** – ubrania zamiast cyfr (również 28×28)
* **CIFAR-10 / CIFAR-100** – kolorowe obrazki (32×32, 10 lub 100 klas)
* **IMDB** – recenzje filmów (do NLP, sentyment analysis)
* **Reuters** – krótkie teksty wiadomości (kategoryzacja tekstu)
* **Boston Housing** – ceny mieszkań w Bostonie (regresja)

---

### 🔹 Przykład: MNIST (klasyczny dataset obrazów)

In [8]:
# Wczytanie danych
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

# Normalizacja (opcjonalnie)
X_train = X_train.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
X_train shape: (60000, 28, 28)
y_train shape: (60000,)


---

### 🔹 Przykład: CIFAR-10 (kolorowe obrazki)

In [6]:
(X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()

print("Obrazki:", X_train.shape)
print("Etykiety:", y_train.shape)

NameError: name 'keras' is not defined

In [7]:
### 🔹 Przykład: IMDB (analiza sentymentu)

In [8]:
# Ograniczamy słownik do 10k najczęstszych słów
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data(num_words=10000)

print("Liczba próbek w train:", len(X_train))
print("Pierwsza próbka (zakodowana):", X_train[0][:10])
print("Etykieta:", y_train[0])

Liczba próbek w train: 25000
Pierwsza próbka (zakodowana): [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]
Etykieta: 1


👉 W tym przypadku dane są już ztokenizowane – jako sekwencje ID słów.




---

# 🚀 Podsumowanie

* **TensorFlow** → używamy `tf.data.experimental.make_csv_dataset` do CSV.
* **Pandas + TensorFlow** → większa kontrola nad preprocessingiem, łatwość użycia.
* **Keras** → dane można przygotować w NumPy/Pandas i przekazać do `model.fit()`.
* **Zaawansowane** → `tf.data.Dataset` daje największą elastyczność (batching, shuffling, mapowanie transformacji).

---

👉 Chcesz żebym zrobił Ci gotowy **notebook Jupyter** z przykładami na `iris.csv` (ładowanie różnymi metodami + prosty model Keras), żebyś mógł od razu użyć na szkoleniu?